In [1]:
import pandas as pd
from sys import maxsize
from itertools import permutations
import requests
import json
import csv
import math
import numpy as np
import folium

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

  ######################################################################################################################################
  ######################################################################################################################################

  #                                               First module : Consignment segregation using Hashing for each locality 
  
  ######################################################################################################################################
  ######################################################################################################################################


# The coordinate of a point is generally given by [Lattitude,Longitude]

# This module takes the list of condignments as an input and segregates the consignments locality wise


df = pd.read_csv('/content/consfinal11.csv')

df.sort_values(["Pincode"],
               axis=0,
               ascending=[True],
               inplace=True)
df.to_csv('consfinal111.csv')
locality = list(set(df.Locality))
your_list=[]
for i in range(len(locality)):
  locality[i] = locality[i].lower()
  #print(locality[i])
  # print(locality) 

for i in range(len(locality)):
   df = pd.read_csv('/content/consfinal111.csv')
   df.drop(df[df['Locality'] != locality[i]].index,inplace=True)
   df.to_csv(locality[i]+'.csv')

   number_lines = df.shape[0]
   p=0
   flag=1
   m=0
   while flag==1:
     p=p+1
     df_for10 = pd.read_csv('/content/'+locality[i]+'.csv',skiprows=range(1,m), nrows=10)
     df_for10.to_csv(locality[i] +str(p)+ '.csv')
     m=m+10
     your_list.append(locality[i] +str(p))
     if(m>=number_lines):
       flag=0

print(your_list)
   #if number_lines>=10:

   #print(number_lines)
   #df.to_csv(locality[i]+'.csv')

['vidya nagar1', 'vidya nagar2', 'vikas nagar1', 'akshay colony1', 'ravi nagar1']


In [3]:



  ######################################################################################################################################
  ######################################################################################################################################

  #                                                 Third module : Travelling Salesman Problem functions 
  
  ######################################################################################################################################
  ######################################################################################################################################


# Here we declare the travelling saleman functions
#This function helps in finding the minimum cost to travel
def tsp_min_cost(cost_matrix, s):
  # s is the starting node
  vertex = []
  for i in range(nodes):
    if i!=s:
      vertex.append(i)
  min_path = maxsize
  next_premutation = permutations(vertex)
  for i in next_premutation:
    curr = 0
    k = s
    for j in i:
      curr += cost_matrix[k][j]
      k = j
    curr += cost_matrix[k][s]
    min_path = min(min_path,curr)
  return min_path



# This function helps to find the path with a target cost path
def tsp_path(cost_matrix, s, target):
  # s is the startinf node
  # target is the target cost
  vertex = []
  ans = []
  for i in range(nodes):
    if i!=s:
      vertex.append(i)
  min_path = maxsize
  next_premutation = permutations(vertex)
  for i in next_premutation:
    curr = 0
    curr_seq = []
    k = s
    curr_seq.append(k+1)
    for j in i:
      curr += cost_matrix[k][j]
      k = j
      curr_seq.append(k+1)
    curr += cost_matrix[k][s]
    curr_seq.append(s+1)
    if curr == target:
      # if the current distance is the shortest distance then update the sequence
      ans = curr_seq
      return ans

In [4]:

######################################################################################################################################
######################################################################################################################################

#                                                 Second module : Construction of distance matrix
  
######################################################################################################################################
######################################################################################################################################



# In this module we take the csv file of each locality and make two API calls : 
# 1. To get the coordinates
# 2. To calculate the shortest path
# Output : Cost matrix / diatance matrix



for k in range(len(your_list)):
  strb='/content/'
  stri = your_list[k]
  input_csv = strb+ stri +'.csv' 

  # reading input file
  list_of_consignments = pd.read_csv(input_csv)

  # retrieving addresses from input csv
  address_list = list(list_of_consignments.Address)

  # to store the values of the coordinates
  list_of_coordinates = []

  office_address = "947F+XFV, Prabhat Colony, Vidya Nagar, Hubli, Karnataka 580031"
  address_list.insert(0,office_address)
  #getting coordinates of addresses
  for j in range(len(address_list)):  
      # strings for using gecoding API request

      str0 = "http://dev.virtualearth.net/REST/v1/Locations/IN/"
      str1 = address_list[j]
      str2 = "?key=AvvVO54Gz52eDy8tWidMYeuyeC4WxGyAnzwJArfFZG1Dqq9mcds9dd8OK3bKgw5E"
      str3 = str0 + str1 + str2

      # making the API call to get the coordinates
      response = requests.get(str3)
      json_object = response.json()

      # the json object that we get as an response has attributes in the following fashion resourceSets -> resources -> point -> coordinates
      dat1 = json_object['resourceSets']
      datt1 = (dat1[0])
      new1 = (datt1['resources'])
      neww1 = new1[0]
      newdata1 = neww1['point']

      #saving all adress coordinates in a list
      list_of_coordinates.append(newdata1['coordinates'])

  #converting address coordinates list to string
  sentence = ""
  for word in list_of_coordinates:
      sentence += str(word) + ";"
  sentence = sentence.replace('[', '')
  sentence = sentence.replace(']', '')
  sentence = sentence.replace(' ', '')
  sentence = sentence.rstrip(sentence[-1])

  # print(list_of_coordinates)
  # strings to make get request for API call
  strr0 = "https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins=" + sentence
  strr1 = "&destinations=" + sentence
  strr2 = "&travelMode=driving&key=AvvVO54Gz52eDy8tWidMYeuyeC4WxGyAnzwJArfFZG1Dqq9mcds9dd8OK3bKgw5E"
  strr3 = strr0 + strr1 + strr2

  # Calling the API for calculating the distance matrix
  response = requests.get(strr3)
  json_object = response.json()

  # the json object that we get as an response has attributes in the following fashion resourceSets -> resources -> results -> travelDistance
  dat = json_object['resourceSets']
  datt = (dat[0])
  new = (datt['resources'])
  neww = new[0]
  newdata = neww['results']

  #total distance enteries
  x = len(newdata)

  #no. of rows and columns of cost matrix
  nodes = int(math.sqrt(x))

  #converting 1D distance array to 2D cost matrix
  temp = np.reshape([d['travelDistance'] for d in newdata],(nodes,nodes) )
  #saving result as csv file
  np.savetxt("newdata.csv",temp, delimiter=",", fmt='%s')
  print("Cost matrix for  : ",your_list[k])
  print(temp)
  # In this module we take the distance matrix and apply travelling salesman problem on the distance matrix
  # Output : Cost matrix / diatance matrix
  cost_matrix = temp
  s = 0

  # first we need to find the minimum cost to traverse the graph
  min_path_cost = tsp_min_cost(cost_matrix,s)
  sequence = []
  s = 0

  # we now find the path with the minimum cost value
  sequence = tsp_path(cost_matrix,s,min_path_cost)
  print("The order in which you should travel is : ",sequence)
  
  
  # After we found out the sequence we store the values of the coodrinates in which it needs to travel in a new list using the sequqnce 
  # given by the travelling salesman functions

  new_coordinate_sequence = []
  for i in range(len(sequence)):
    new_coordinate_sequence.append(list_of_coordinates[sequence[i]-1])
  

  ######################################################################################################################################
  ######################################################################################################################################

  #                                                 Fourth module : Vizualizer 
  
  ######################################################################################################################################
  ######################################################################################################################################
  
  # # This array is for different colors of the coordinate points
  # colors = ["red","green","blue","red","green","blue","red","green","blue","red","green","blue","red","green","blue","red","green","blue","red","green","blue"]

  # location = (new_coordinate_sequence[0][0],new_coordinate_sequence[0][1])
  # # find shortest route based on the mode of travel
  # mode = 'bike'        # 'drive', 'bike', 'walk'
  # # find shortest path based on distance or time
  # optimizer = 'length'        # 'length','time'
  # # create graph from OSM within the boundaries of some 
  # # geocodable place(s)
  # G = ox.graph_from_point(location, network_type=mode)
  # nodes, edges = ox.graph_to_gdfs(G)
  # orgi_node = []
  # dest_node = []
  # shortest_route = []

  # for z in range(len(new_coordinate_sequence)-1):
  #   start_latlng = (new_coordinate_sequence[z][0], new_coordinate_sequence[z][1])
  #   end_latlng = (new_coordinate_sequence[z+1][0], new_coordinate_sequence[z+1][1])
  #   orgi_node_tmp = ox.distance.get_nearest_node(G, start_latlng)
  #   dest_node_tmp = ox.distance.get_nearest_node(G, end_latlng)
  #   print(orgi_node_tmp)
  #   print(dest_node_tmp)
  #   shortest_route_tmp = nx.shortest_path(G,orgi_node_tmp,dest_node_tmp,weight=optimizer)
  #   orgi_node.append(orgi_node_tmp)
  #   dest_node.append(dest_node_tmp)
  #   shortest_route.append(shortest_route_tmp)
  # shortest_route_map = ox.plot_route_folium(G, shortest_route[0],tiles='openstreetmap')
  # for z in range(1,len(shortest_route)):
  #   shortest_route_map = ox.plot_route_folium(G, shortest_route[z],tiles='openstreetmap')
  # shortest_route_map
  # print(shortest_route)
  # new_coordinate_sequence.clear()
  # orgi_node.clear()
  # dest_node.clear()
  # shortest_route.clear()

  x = 0
  m=folium.Map(location=new_coordinate_sequence[0],zoom_start=15)
  last = [0,0]
  for each in new_coordinate_sequence:
    folium.Marker(each,popup=str(x),tooltip='<strong>Click here to see order</strong>',icon=folium.Icon(color='red',icon='none')).add_to(m)
    x = x+1
    if last == each:
      x = x-1
    last = each
  #add lines
  folium.PolyLine(new_coordinate_sequence, color="red", weight=2.5, opacity=1).add_to(m)
  start_marker = folium.Marker(location = new_coordinate_sequence[0],popup='Office',tooltip='<strong>Click here to see Popup</strong>',icon = folium.Icon(color='green'))
  start_marker.add_to(m)
  display(m)


Cost matrix for  :  vidya nagar1
[[0.    3.94  3.94  3.94  2.067 0.    1.921 0.    0.904 1.276 3.237]
 [3.448 0.    0.    0.    4.918 3.448 4.13  3.448 3.755 4.127 3.603]
 [3.448 0.    0.    0.    4.918 3.448 4.13  3.448 3.755 4.127 3.603]
 [3.448 0.    0.    0.    4.918 3.448 4.13  3.448 3.755 4.127 3.603]
 [3.276 6.559 6.559 6.559 0.    3.276 2.968 3.276 3.523 3.81  5.211]
 [0.    3.94  3.94  3.94  2.067 0.    1.921 0.    0.904 1.276 3.237]
 [1.984 4.005 4.005 4.005 3.394 1.984 0.    1.984 2.231 2.603 4.209]
 [0.    3.94  3.94  3.94  2.067 0.    1.921 0.    0.904 1.276 3.237]
 [0.828 3.426 3.426 3.426 2.298 0.828 2.75  0.828 0.    0.59  2.148]
 [1.144 3.798 3.798 3.798 2.614 1.144 3.066 1.144 0.59  0.    2.092]
 [2.679 3.161 3.161 3.161 4.447 2.679 4.205 2.679 1.861 1.626 0.   ]]
The order in which you should travel is :  [1, 5, 7, 2, 3, 4, 11, 10, 9, 6, 8, 1]


Cost matrix for  :  vidya nagar2
[[0.    3.237 3.305 0.78  4.376 1.276 7.91  0.    0.    0.    3.362]
 [2.679 0.    2.036 2.884 4.182 1.626 2.47  2.679 2.679 2.679 0.167]
 [2.523 1.892 0.    2.43  6.302 2.285 5.819 2.523 2.523 2.523 2.017]
 [0.907 3.283 2.444 0.    4.56  1.46  7.475 0.907 0.907 0.907 3.408]
 [2.244 4.191 4.892 3.025 0.    2.778 4.822 2.244 2.244 2.244 4.09 ]
 [1.144 2.092 3.163 1.502 4.923 0.    4.199 1.144 1.144 1.144 2.217]
 [7.324 2.47  5.939 8.105 4.813 3.849 0.    7.324 7.324 7.324 2.339]
 [0.    3.237 3.305 0.78  4.376 1.276 7.91  0.    0.    0.    3.362]
 [0.    3.237 3.305 0.78  4.376 1.276 7.91  0.    0.    0.    3.362]
 [0.    3.237 3.305 0.78  4.376 1.276 7.91  0.    0.    0.    3.362]
 [2.816 0.167 2.161 3.021 3.918 1.763 2.295 2.816 2.816 2.816 0.   ]]
The order in which you should travel is :  [1, 4, 3, 2, 11, 7, 5, 6, 8, 9, 10, 1]


Cost matrix for  :  vikas nagar1
[[0.    2.419 3.688 2.837 4.732 4.069]
 [2.085 0.    1.681 0.833 2.568 4.318]
 [2.067 1.02  0.    1.438 2.578 5.338]
 [2.209 0.738 1.344 0.    2.391 4.808]
 [4.321 2.633 2.549 2.391 0.    4.544]
 [5.495 4.024 5.091 4.243 3.96  0.   ]]
The order in which you should travel is :  [1, 6, 5, 3, 2, 4, 1]


Cost matrix for  :  akshay colony1
[[0.    2.076 2.076 0.897 7.875 0.   ]
 [2.122 0.    0.    2.378 4.786 2.122]
 [2.122 0.    0.    2.378 4.786 2.122]
 [0.958 2.377 2.377 0.    7.059 0.958]
 [7.24  4.306 4.306 7.066 0.    7.24 ]
 [0.    2.076 2.076 0.897 7.875 0.   ]]
The order in which you should travel is :  [1, 4, 5, 2, 3, 6, 1]


Cost matrix for  :  ravi nagar1
[[0.    1.697 2.323 2.674 2.713 3.14  0.    2.138]
 [1.849 0.    0.625 0.976 1.015 1.442 1.849 0.44 ]
 [2.49  0.93  0.    0.377 0.498 0.843 2.49  0.259]
 [2.735 1.065 0.374 0.    0.181 0.465 2.735 0.475]
 [2.856 1.296 0.498 0.304 0.    0.367 2.856 0.599]
 [4.043 1.39  0.84  0.465 0.367 0.    4.043 0.941]
 [0.    1.697 2.323 2.674 2.713 3.14  0.    2.138]
 [2.459 0.899 0.209 0.478 0.599 0.944 2.459 0.   ]]
The order in which you should travel is :  [1, 2, 8, 3, 4, 6, 5, 7, 1]
